In [ ]:
import numpy as np 
import pandas as pd 
import pickle
import matplotlib.pyplot as plt
from timeit import default_timer as timer

from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, AvgPool2D, BatchNormalization, Reshape
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler

import os
for dirname, _, filenames in os.walk('../input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

print(os.listdir('../input'))

print()
print(os.listdir('../usr/lib'))



In [ ]:
import utility_scripts_for_traffic_signs


print(help(utility_scripts_for_traffic_signs))


print(dir(utility_scripts_for_traffic_signs))


In [ ]:
n = [2, 3, 7, 8]


m = ['RGB + /255 + Mean', 'RGB + /255 + Mean + STD', 'LHE + /255 + Mean', 'LHE + /255 + Mean + STD']


data = {}


for i in n:
   
    with open('../input/traffic-signs-preprocessed/data' + str(i) + '.pickle', 'rb') as f:
        data[i] = pickle.load(f, encoding='latin1')  

    data[i]['y_train'] = to_categorical(data[i]['y_train'], num_classes=43)
    data[i]['y_validation'] = to_categorical(data[i]['y_validation'], num_classes=43)

   
    data[i]['x_train'] = data[i]['x_train'].transpose(0, 2, 3, 1)
    data[i]['x_validation'] = data[i]['x_validation'].transpose(0, 2, 3, 1)
    data[i]['x_test'] = data[i]['x_test'].transpose(0, 2, 3, 1)


ii = 0  
for i in n:
    print('data' + str(i) + '.pickle ->', m[ii])
    for k, j in data[i].items():
        if k == 'labels':
            print(k + ':', len(j))
        else: 
            print(k + ':', j.shape)
    print()
    ii += 1


In [ ]:
%matplotlib inline


some_examples = data[2]['x_train'][:49, :, :, :]
print(some_examples.shape)  # (49, 32, 32, 3)


fig = plt.figure()
grid = utility_scripts_for_traffic_signs.convert_to_grid(some_examples)
plt.imshow(grid.astype('uint8'))
plt.axis('off')
plt.gcf().set_size_inches(15, 15)
plt.title('Some training examples', fontsize=18)
plt.show()
plt.close()


fig.savefig('some_training_examples.png')
plt.close()


In [ ]:
%matplotlib inline

with open('../input/traffic-signs-preprocessed/data0.pickle', 'rb') as f:
        data0 = pickle.load(f, encoding='latin1')  # dictionary type


data0['x_train'] = data0['x_train'].transpose(0, 2, 3, 1)


example0 = data0['x_train'][2, :, :, :]
print(example0.shape)  # (32, 32, 3)


example = {}


example[2] = data[2]['x_train'][2, :, :, :]
example[3] = data[3]['x_train'][2, :, :, :]
print(example[2].shape, example[3].shape)  # (32, 32, 3) (32, 32, 3)


example[7] = data[7]['x_train'][2, :, :, 0]
example[8] = data[8]['x_train'][2, :, :, 0]
print(example[7].shape, example[8].shape)  # (32, 32) (32, 32)




labels = []


r = pd.read_csv('../input/traffic-signs-preprocessed/label_names.csv')

for name in r['SignName']:
    
    labels.append(name)




plt.rcParams['figure.figsize'] = (12.0, 12.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['font.family'] = 'Times New Roman'

fig = plt.figure()


plt.subplot(2, 4, 1)  
plt.imshow(example0)
plt.xlabel('RGB', fontsize=15)
plt.ylabel(labels[np.argmax(data[2]['y_train'][2])], fontsize=15)
plt.xticks([])
plt.yticks([])


k = 5  
ii = 0  
for i in n:
    plt.subplot(2, 4, k) 
    if i == 7 or i == 8:
        plt.imshow(example[i], cmap='gray')
    else:
        plt.imshow(example[i])
    plt.xlabel(m[ii], fontsize=15)
    plt.ylabel(labels[np.argmax(data[2]['y_train'][2])], fontsize=15)
    plt.xticks([])
    plt.yticks([])
    k += 1
    ii += 1


plt.subplots_adjust(hspace=0)
plt.tight_layout()


plt.show()


fig.savefig('one_ts_from_different_datasets.png')
plt.close()


In [ ]:
model = Sequential()

model.add(Conv2D(32, kernel_size=9, padding='same', activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPool2D(pool_size=2))

model.add(Conv2D(64, kernel_size=7, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2))

model.add(Conv2D(128, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2))

model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dense(43, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** (x + epochs))
epochs = 20

h = model.fit(data[2]['x_train'][:100], data[2]['y_train'][:100],
              batch_size=5, epochs = epochs,
              validation_data = (data[2]['x_validation'], data[2]['y_validation']),
              callbacks=[annealer], verbose=1)

print()
print('Epochs={0:d}, Train accuracy={1:.5f}, \
      Validation accuracy={2:.5f}'.\
      format(epochs, max(h.history['accuracy']), max(h.history['val_accuracy'])))


In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (15.0, 5.0) # Setting default size of plots
plt.rcParams['image.interpolation'] = 'nearest'

fig = plt.figure()

plt.plot(h.history['accuracy'], '-o', linewidth=3.0)
plt.plot(h.history['val_accuracy'], '-o', linewidth=3.0)
plt.title('Overfitting small data for RGB dataset[2]', fontsize=22)
plt.legend(['train', 'validation'], loc='upper left', fontsize='xx-large', borderpad=2)
plt.xlabel('Epoch', fontsize=22)
plt.ylabel('Accuracy', fontsize=22)
plt.tick_params(labelsize=18)
plt.show()

# Saving the plot
fig.savefig('Overfitting_dataset_2.png')
plt.close()


# Building set of models of CNN with Keras

In [ ]:
# Defining dictionary for models
model = {}

# Building four models
for i in n:
    model[i] = Sequential()
    
    if i == 7 or i == 8:
        model[i].add(Conv2D(32, kernel_size=9, padding='same', activation='relu', input_shape=(32, 32, 1)))
    else:
        model[i].add(Conv2D(32, kernel_size=9, padding='same', activation='relu', input_shape=(32, 32, 3)))
        
    model[i].add(MaxPool2D(pool_size=2))

    model[i].add(Conv2D(64, kernel_size=7, padding='same', activation='relu'))
    model[i].add(MaxPool2D(pool_size=2))

    model[i].add(Conv2D(128, kernel_size=3, padding='same', activation='relu'))
    model[i].add(MaxPool2D(pool_size=2))

    model[i].add(Flatten())
    model[i].add(Dense(500, activation='relu'))
    model[i].add(Dense(43, activation='softmax'))

    model[i].compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
for i in n:
    name = 'model-dataset' + str(i) + '.h5'
    model[i].save(name)

